<h1>JetCat P300-Pro Stock Cycle Analysis<h1>

Package Import

In [1]:
import math as m
import numpy as np
import pandas as pd


<h2>Metric Calculations<h2>

<h3>Metric Characteristics<h3>

In [2]:
pressure_ratio = 3.55
mdot_stock = 0.5
stock_max_fuel_consumption_m = .980 # L/min
stock_max_fuel_consumption_s = stock_max_fuel_consumption_m/60 # L/s
stock_idle_fuel_consumption_m = 0.179 # L/min
stock_idle_fuel_consumption_s = stock_idle_fuel_consumption_m/60 # L/s
engine_mass = 2.730 # kg
min_exhaust_gas_temp = 480+273.15 # K
max_exhaust_gas_temp = 750+273.15 # K
idle_rpm = 35000 # (1/min)
max_rpm = 105000 # (1/min)
idle_thrust = 14 #N
max_stock_thrust = 300 #N
max_reported_exhaust_gas_velocity = (2160 * (1/3600) * 1000) # m/s
exhaust_gas_power_output = (90 * 1000)
max_sfc = 0.157 # (kg/Nh)
Tmax = 993 # K

# https://www.jetcat.de/en/productdetails/produkte/jetcat/produkte/Professionell/p300%20pro

<h3>Metric Dimensions<h3>

In [3]:
starter_to_nozz_exit_length = .381
entrance_to_nozz_exit_length = starter_to_nozz_exit_length-.080
casing_width = .132
casing_width_with_mounts = .188

# nozzle
turbine_exit_outer_diameter = .091
nozzle_length = .104

# Entrance - Estimated
entrance_ID = .120 #m
entrance_A = (m.pi*((entrance_ID/2)**2))



<h4>Ambient Conditions<h4>

In [4]:
Pa = 101325 # Pa
Ta = 298 # K
rho_a = 1.225 # kg/m^3
R = 287.7 # J/(kg*K)
Qr = (43.1)*(10**6) # Lower Heating Value of Kerosene

# Make below variables dynamically calculate later on 
# --------------------------------------------------
Gc = 1.4 # Freestream Gamma, ratio of specific heats
Gh = 1.35 # Hot air Gamma, ratio of specific heats
Cpc = ((Gc)/(Gc-1))*R # Cp for freestream gamma
Cph = ((Gh)/(Gh-1))*R # Cp for compressed gamma
# --------------------------------------------------


Ambient through Inlet

In [5]:
u1 = mdot_stock/(rho_a*entrance_A)
a1 = m.sqrt(Ta*Gc*R)
M1 = u1/a1
To1 = Ta + ((u1**2)/(2*Cpc)) # Stagnation Temperature in the Inlet
Po1 = Pa*(1+((u1**2)/(2*Cpc*Ta)))**((Gc)/(Gc-1)) # Stagnation Pressure in the Inlet

<h4>Inlet to Compressor<h4>

In [6]:
# Conservation of mass 
u2 = mdot_stock/(rho_a*entrance_A)

Nc = np.linspace(.5, 1, 1000) # Guess Compressor Efficiency 
Po2 = Pa * pressure_ratio
To2 =  (1+(1/Nc)*(pressure_ratio**((1.4-1)/(1.4))-1))*To1

Compressor Through Combustor

In [7]:
f = .000144 # fuel/air mixture ratio
Nm = np.linspace(.5, 1, 1000) # Generate 1000 Mechanical Efficiency Values
Nb = np.linspace(.5, 1, 1000) # Generate 1000 Combustor Efficiency Values
To3 = ((Cpc*To2)+(Nb*f*Qr))/((1+f)*Cph) # Stagnation Temperature in the Combustor

In [ ]:
fcombustion = abs(((To3/To2)-1)/(((Nb*Qr)/(Cph*To2))-(To3/To2))) # Fuel/air ratio for fuel burning
fcombustion = abs(((Tmax/To2)-1)/(((Nb*Qr)/(Cph*To2))-(Tmax/To2)))
To3 = ((Cpc*To2)+(Nb*fcombustion*Qr))/((1+fcombustion)*Cph) # Combustor Temperature after heat addition from fuel combusting
Pb = np.linspace(0, .10, 1000); # Generate 1000 Values for Pressure loss in the combustor 
Po3 = Po2 - (1*(Po2*Pb)) # Stagnation Pressure with loss in the combustor

Combustor Through Turbine

In [ ]:
To4 = Tmax + ((Cpc/Cph)*(To3-To2)) # Stagnation Temperature in the Turbine
To4Actual = To4-To2 # This correction here results in a correct outlet temperature. The above equation results in a temperature much higher than the combustor. This is also concistent with the Inconel 718 temperature being rated to around 993 degrees Kelvin
To4Prime = To3-(To3-To4) # 
Po4 = Po3*((To4Actual/To3)**((Gh)/(Gh-1))) # The Stagnation Pressure in the turbine 
Po4Pa = Po4/Pa # Check Critical Pressure Ratio 

Turbine Through Nozzle

In [ ]:
T5 = (2/(Gh+1))*To4Actual # Static Temperature at Nozzle Exit

P5 = Po4*(1./Po4Pa) # Static Pressure at Nozzle Exit

a5 = m.sqrt(Gh*R*T5) # Mach velocity at Nozzle Exit

AreaBetweenInnerAndOuter = ((m.pi/4)*(.049**2))-(m.pi/4*(.03733**2)) # Close to Averaged Calculation between inner cone and outer nozzle shell

u5 = (mdot_stock/(rho_a*AreaBetweenInnerAndOuter)) # Exit Velocity at the nozzle

A5overMdot = (m.pi*((.00762/2)**2))/(mdot_stock) # Exit Area / Mass Flow 

ThrustFinal = (((1+fcombustion)*u5)-u1) + (A5overMdot*(P5-Pa)) # Thrust Calculation